In [47]:
import sklearn
import mne
import numpy as np
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


to była moja pierwsz myśl, że tak nalezy zrobić, aleee wtedy zostało nam 50 próbek, dlatego wydaje mi się, że nie powinnismy robić tutaj tego uśrednienia.

In [46]:
# # to była moja pierwsz myśl, że tak nalezy zrobić, aleee wtedy zostało nam 50 próbek, dlatego wydaje mi się, że nie powinnismy robić tutaj tego uśrednienia.
#
# data_path = r"D:\studia\magisterka\dane EEG\BADANIE_POLITYCZNE_2022_eeg_bdfy\EEG_preprocessed"
#
# file_list = sorted(glob.glob(os.path.join(data_path, "s*.bdf-epo.fif")))
#
# evoked_list_up = []
# evoked_list_inv = []
#
# for file_path in file_list:
#     epochs = mne.read_epochs(file_path, preload=True)
#
#     evoked_up = epochs["up"].average() ## CZY TO JEST KONIECZNE??
#     evoked_up = np.array(evoked_up.get_data())
#     evoked_list_up.append(evoked_up)
#
#
#     evoked_inv = epochs["inv"].average() ## CZY TO JEST KONIECZNE??
#     evoked_inv = np.array(evoked_inv.get_data())
#     print(evoked_inv.shape)
#     evoked_list_inv.append(evoked_inv)
#
# evoked_list_up = np.array(evoked_list_up)
# evoked_list_inv = np.array(evoked_list_inv)
#
# print(evoked_list_up.shape, evoked_list_inv.shape)

In [43]:
def load_eeg_epochs(data_path, picks=None, t_min = -0.2, t_max = 0.5):
    file_list = sorted(glob.glob(os.path.join(data_path, "s*.bdf-epo.fif")))

    epochs_list_up = []
    epochs_list_inv = []

    for file_path in file_list:
        with mne.utils.use_log_level("ERROR"):
            epochs = mne.read_epochs(file_path, preload=True)

            epochs_up = epochs["up"]
            epochs_up = np.array(epochs_up.get_data(picks=picks, tmin=t_min, tmax=t_max))
            for i in range(epochs_up.shape[0]):
                epochs_list_up.append(epochs_up[i])


            epochs_inv = epochs["inv"]
            epochs_inv = np.array(epochs_inv.get_data(picks=picks, tmin=t_min, tmax=t_max))
            for i in range(epochs_inv.shape[0]):
                epochs_list_inv.append(epochs_inv[i])

    epochs_list_up = np.array(epochs_list_up)
    epochs_list_inv = np.array(epochs_list_inv)

    epochs_list_up = epochs_list_up.reshape(epochs_list_up.shape[0], -1)
    epochs_list_inv = epochs_list_inv.reshape(epochs_list_inv.shape[0], -1)

    labels_up = [0] * len(epochs_list_up)
    labels_inv = [1] * len(epochs_list_inv)

    X = np.concatenate((epochs_list_up, epochs_list_inv), axis=0)
    Y = np.concatenate((labels_up, labels_inv), axis=0)

    return X, Y

In [48]:
def train_and_test_model(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        Y,
        test_size=0.2,
        random_state=42
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = LogisticRegression(max_iter=10000)
    model.fit(X_train, y_train)

    # predict test data
    y_test_pred = model.predict(X_test)
    test_score = accuracy_score(y_test, y_test_pred)

    # predict train data
    y_train_pred = model.predict(X_train)
    train_score = accuracy_score(y_train, y_train_pred)

    print(f"test_score: {test_score:.4f}")
    print(f"train_score: {train_score:.4f}")


In [49]:
data_path = r"D:\studia\magisterka\dane EEG\BADANIE_POLITYCZNE_2022_eeg_bdfy\EEG_preprocessed"
picks_1 = ["P3", "P4", "P7", "P8", "P9", "P10", "O1", "O2"]
picks_2 = ["P7", "P8"]

X, Y = load_eeg_epochs(data_path)
train_and_test_model(X, Y)

X, Y = load_eeg_epochs(data_path, picks_1)
train_and_test_model(X, Y)

X, Y = load_eeg_epochs(data_path, picks_2)
train_and_test_model(X, Y)

test_score: 0.7430
train_score: 0.9998
test_score: 0.7336
train_score: 0.7675
test_score: 0.6981
train_score: 0.7190
